TODO:
- Double check everything
- Polish

# Demo for the Fine-Tuning LLMs for Text Analysis workshop

Note: the code in this notebook is for demonstration purposes. You need to adapt it for a research project. The hyperparameters used here are also not optimal.

## Install libraries required for the workshop and that are not available by default in Google Colab

In [1]:
! pip install evaluate

## Import libraries

In [2]:
# To work with dataframes
import pandas as pd

# To work with arrays
import numpy as np

# To split data
from sklearn.model_selection import train_test_split

# To prepare the data in the expected format
from datasets import DatasetDict, Dataset

# To fine-tune model
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# To evaluate model
import evaluate

# To select the computing device and other uses of PyTorch
import torch

## Check whether we're using CPU or GPU

CUDA is a parallel computing platform and API developed by NVIDIA to use GPUs. In the code chunk below, if it says "cuda", it means we're using a GPU.

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Read the data

In [4]:
train = pd.read_csv("https://raw.githubusercontent.com/MoritzLaurer/less-annotating-with-bert-nli/refs/heads/master/data_clean/df_manifesto_protectionism_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/MoritzLaurer/less-annotating-with-bert-nli/refs/heads/master/data_clean/df_manifesto_protectionism_test.csv")

In [5]:
train.shape

(2116, 16)

In [6]:
train.head(1)

,idx,label,label_text,text_original,label_domain_text,label_subcat_text,text_preceding,text_following,manifesto_id,doc_id,country_name,date,party,cmp_code_hb4,cmp_code,label_subcat_text_simple
0,85699,0,Other,People are understandably concerned by the lac...,Social Groups,Agriculture and Farmers: Positive,The Australian Bureau of Statistics’ small vol...,It also means the Foreign Investment Review Bo...,63810_201309,95,Australia,201309,63810,703,703.0,Agriculture and Farmers: Positive


In [7]:
train['label'].value_counts()

,count
label,
0,1058
1,564
2,494


In [8]:
train['label_text'].value_counts()

,count
label_text,
Other,1058
Protectionism: Negative,564
Protectionism: Positive,494


Here we're going to focus on negative vs. positive for simplicity and speed.

In [9]:
test.shape

(3762, 16)

In [10]:
test.head(1)

,idx,label,label_text,text_original,label_domain_text,label_subcat_text,text_preceding,text_following,manifesto_id,doc_id,country_name,date,party,cmp_code_hb4,cmp_code,label_subcat_text_simple
0,104942,0,Other,We will now integrate the efforts of governmen...,Political System,Governmental and Administrative Efficiency,The National-led Government has developed new ...,and develop an aggressive approach to attracti...,64620_201111,131,New Zealand,201111,64620,303,303.0,Governmental and Administrative Efficiency


In [11]:
test['label'].value_counts()

,count
label,
0,3420
1,172
2,170


In [12]:
test['label_text'].value_counts()

,count
label_text,
Other,3420
Protectionism: Negative,172
Protectionism: Positive,170


## Clean data

Drop "Other":

In [13]:
train = train[train['label'] != 0]
test = test[test['label'] != 0]

In [14]:
train['label'].value_counts()

,count
label,
1,564
2,494


In [15]:
test['label'].value_counts()

,count
label,
1,172
2,170


Label 1 as 0 and 2 as 1. This is important because otherwise later, in `trainer.train()`, you'll get an error (`CUDA error: device-side assert triggered`) because `CrossEntropyLoss`, which is used by defaul in `Trainer` for classificaton, expects class labels starting at 0 (see [here](https://stackoverflow.com/questions/51691563/cuda-runtime-error-59-device-side-assert-triggered) and [here](https://drdroid.io/stack-diagnosis/pytorch-runtimeerror--cuda-error--device-side-assert-triggered)).

In [16]:
train['label'] = train['label'] - 1
test['label'] = test['label'] - 1

/tmp/ipython-input-16-956503915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['label'] = train['label'] - 1
/tmp/ipython-input-16-956503915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['label'] = test['label'] - 1


In [17]:
train['label'].value_counts()

,count
label,
0,564
1,494


In [18]:
test['label'].value_counts()

,count
label,
0,172
1,170


Put together all the text (preceding, original, and following):

In [19]:
train['text'] = train['text_preceding'] + " " + train['text_original'] + " " + train['text_following']
test['text'] = test['text_preceding'] + " " + test['text_original'] + " " + test['text_following']

/tmp/ipython-input-19-1945906953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text'] = train['text_preceding'] + " " + train['text_original'] + " " + train['text_following']
/tmp/ipython-input-19-1945906953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['text'] = test['text_preceding'] + " " + test['text_original'] + " " + test['text_following']


In [20]:
train[['text_preceding', 'text_original', 'text_following', 'text']].head(1)

,text_preceding,text_original,text_following,text
1058,Support and develop the ‘buy New Zealand-made’...,"and, where practicable place ‘buy New Zealand’...",Review all current and future bilateral Free T...,Support and develop the ‘buy New Zealand-made’...


In [21]:
test[['text_preceding', 'text_original', 'text_following', 'text']].head(1)

,text_preceding,text_original,text_following,text
3420,We see the Asia Pacific Economic Council (APEC...,• Work to ensure that New Zealand is not disad...,• Aim at more than doubling our total research...,We see the Asia Pacific Economic Council (APEC...


Remove missing values:

In [22]:
train = train[train['text'].notna()]

Here's what we have as a reminder:

In [23]:
print(train.loc[1058, 'label_text'])
print(train.loc[1058, 'label'])
print(train.loc[1058, 'text'])
print("")
print(train.loc[2111, 'label_text'])
print(train.loc[2111, 'label'])
print(train.loc[2111, 'text'])

Protectionism: Positive
1
Support and develop the ‘buy New Zealand-made’ campaign and, where practicable place ‘buy New Zealand’ purchasing requirements on taxpayer and ratepayer owned businesses and State Owned Enterprises. Review all current and future bilateral Free Trade Agreements (FTAs), including the Closer Economic Relations (CER) process, to improve transparency and accountability and to ensure they are in New Zealand's interest.

Protectionism: Negative
0
Where we are: Wales risks being totally unrepresented on the international stage, with no Welsh presence in UK embassies to champion our people and businesses. Plaid Cymru's answer: Plaid Cymru will develop a real international policy for Wales, so that we can restore our position as a great trading nation. We will introduce a Welsh Development Agency (WDA) for the 21st century, tasked with boosting Welsh trade.


## Split `test` into `validation`, `test`, and `new`

This is only for demonstration purposes for this workshop.

In [24]:
SEED = 6325

validation, temp = train_test_split(test, test_size=0.5, random_state=SEED)

test, new = train_test_split(temp, test_size=0.5, random_state=SEED)

In [25]:
validation.shape

(171, 17)

In [26]:
validation['label'].value_counts()

,count
label,
0,92
1,79


In [27]:
test.shape

(85, 17)

In [28]:
test['label'].value_counts()

,count
label,
0,43
1,42


In [29]:
new.shape

(86, 17)

In [30]:
new['label'].value_counts()

,count
label,
1,49
0,37


## Convert to [`DatasetDict`](https://huggingface.co/docs/datasets/v3.6.0/en/package_reference/main_classes#datasets.DatasetDict)

This prepares the data in the expected format for training and evaluation.

In [31]:
dataset = DatasetDict({
    'train': Dataset.from_pandas(train[['label', 'text']].reset_index(drop=True)),
    'validation': Dataset.from_pandas(validation[['label', 'text']].reset_index(drop=True)),
    'test': Dataset.from_pandas(test[['label', 'text']].reset_index(drop=True))
})
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1057
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 171
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 85
    })
})

## Set model that we're going to use

Name from the [Hugging Face website](https://huggingface.co/distilbert/distilbert-base-uncased):

In [32]:
MODEL_NAME = "distilbert/distilbert-base-uncased"

## Tokenize the data

Load appropriate tokenizer for the model:

In [33]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Create function to tokenize:

In [34]:
def tokenize(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        # I added max_length for speed.
        # Otherwise it defaults to the max the model can take https://huggingface.co/docs/transformers/en/pad_truncation
        max_length=128
        )

In [35]:
example_input = {"text": ["I love this workshop!"]}

tokenize(example_input)

{'input_ids': [[101, 1045, 2293, 2023, 8395, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

The output is:
- `input_ids`: a list of token IDs corresponding to the input texts. `101` is the CLS token, `1045` is for "I", ..., `102` is SEP, and the rest is padding.
- `attention_mask` is a list indicating which tokens should be attended to, where 1 is a real token and 0 is padding.

Let's tokenize the whole dataset:

In [36]:
# batched = TRUE to operate on batches of examples rather than individual for speed
# https://huggingface.co/docs/datasets/en/process#batch-processing
dataset = dataset.map(tokenize, batched=True)
dataset

Map:   0%|          | 0/1057 [00:00<?, ? examples/s]

Map:   0%|          | 0/171 [00:00<?, ? examples/s]

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1057
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 171
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 85
    })
})

## Specify how many number of categories

In [37]:
NUM_LABELS = len(train['label'].unique())

## Load the model

In [38]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define metrics for evaluation

In [39]:
metrics = evaluate.combine(["accuracy", "precision", "recall", "f1"])

Create function to calculate metrics. We need to create `compute_metrics_closure` because we want `metrics` to be an argument but `Trainer` expects the function to accept only one argument (an instance of `EvalPrediction`).

In [40]:
def compute_metrics_closure(metrics):
  """
  Creates a compute_metrics function with the provided evaluation metrics.

  Args:
      metrics (evaluate.EvaluationModule): A combined metric object from the `evaluate` library.

  Returns:
      function: A function that computes the provided metrics for given model predictions.
  """

  def compute_metrics(eval_pred):
      """
      Computes evaluation metrics for model predictions.

      Args:
          eval_pred (tuple): A tuple containing:
              - logits (np.ndarray): The raw output predictions from the model.
              - labels (np.ndarray): The true labels corresponding to the inputs.
      Returns:
          dict: A dictionary containing the computed metric(s).
      """

      # Unpack the logits and labels from the evaluation prediction tuple
      logits, labels = eval_pred

      # Convert the raw logits to predicted class labels by selecting the index with the highest logit value
      predictions = np.argmax(logits, axis=-1)

      # Compute and return the evaluation metric(s) using the predictions and true labels
      return metrics.compute(predictions=predictions, references=labels)

  return compute_metrics

## Define arguments for training

In [41]:
# https://huggingface.co/transformers/v4.7.0/main_classes/trainer.html#transformers.TrainingArguments
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written.
    output_dir="./output",
    # Total number of training epochs to perform.
    num_train_epochs=1,
    # The batch size per GPU/TPU core/CPU for training.
    per_device_train_batch_size=8,
    # The batch size per GPU/TPU core/CPU for evaluation.
    per_device_eval_batch_size=8,
    # Learning rate
    learning_rate=5e-5,
    # Weight decay
    weight_decay=0.0,
    # The logging strategy to adopt during training. Here, logging is done at the end of each epoch.
    logging_strategy="epoch",
    # The list of integrations to report the results and logs to.
    report_to="none"
)

## Train the model

Define [`Trainer`](https://huggingface.co/docs/transformers/v4.52.3/en/main_classes/trainer#transformers.Trainer) object. This object takes a pretrained model and prepares it for training and evaluation. It abstracts a lot of the complexity that would be necessary using PyTorch directly.

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics_closure(metrics)
)

Train model:

In [43]:
trainer.train()

Step,Training Loss
133,0.470400


TrainOutput(global_step=133, training_loss=0.47037127501982495, metrics={'train_runtime': 22.082, 'train_samples_per_second': 47.867, 'train_steps_per_second': 6.023, 'total_flos': 35004510094848.0, 'train_loss': 0.47037127501982495, 'epoch': 1.0})

- `global_step`: total number of optimization steps (batches) processed.
- `training_loss`: average loss computed over all batches in this training run.
- `metrics`: dictionary with additional stats.
  - `train_runtime`: time taken for training.
  - `train_samples_per_second`: how many samples processed per second.
  - `train_steps_per_second`: steps performed per second.
  - `total_flos`: total floating-point operations used.
  - `train_loss`: average loss.
  - `epoch`: epoch number completed.

## Test the model

In [44]:
trainer.evaluate(eval_dataset=dataset["test"])

{'eval_loss': 0.5025452375411987,
 'eval_accuracy': 0.7647058823529411,
 'eval_precision': 0.7619047619047619,
 'eval_recall': 0.7619047619047619,
 'eval_f1': 0.7619047619047619,
 'eval_runtime': 0.427,
 'eval_samples_per_second': 199.084,
 'eval_steps_per_second': 25.764,
 'epoch': 1.0}

- `eval_loss`: average loss calculated over the evaluation (test) dataset.
- `eval_accuracy`, `eval_precision`, `eval_recall`, `eval_f1`: metrics computed via the `compute_metrics` function.
- `eval_runtime`: time taken to run the evaluation loop.
- `eval_samples_per_second`: how many test samples were processed per second.
- `eval_steps_per_second`: rate at which evaluation batches were processed.
- `epoch`: epoch number at which evaluation was done.

## Generate predictions on new data

Create dataset for `new` data since this is the format that `Trainer` expects.

In [45]:
new_dataset = Dataset.from_pandas(new[['text']].reset_index(drop=True))

Tokenize new data with the same tokenizer used during training.

In [46]:
new_dataset = new_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/86 [00:00<?, ? examples/s]

Use model to get predictions (logits).

In [47]:
predictions = trainer.predict(new_dataset)
predictions

PredictionOutput(predictions=array([[-0.78361255,  0.7879547 ],
       [ 1.3924159 , -1.4180576 ],
       [ 1.1251961 , -1.2252665 ],
       [-0.12519355,  0.17314416],
       [-1.2161793 ,  1.3727537 ],
       [-1.3480221 ,  1.3811079 ],
       [ 1.3010815 , -1.3765541 ],
       [-1.0024374 ,  1.1217505 ],
       [ 1.3942534 , -1.5273594 ],
       [ 0.99308556, -0.96962404],
       [-1.2071494 ,  1.3180724 ],
       [-0.0117905 , -0.05663886],
       [ 0.39861912, -0.5109151 ],
       [-0.23184982,  0.266297  ],
       [-1.0627222 ,  1.201213  ],
       [-0.6160534 ,  0.6682716 ],
       [ 1.0276047 , -1.0495312 ],
       [-0.9789389 ,  1.1112759 ],
       [-0.21778204,  0.30491042],
       [ 1.3858902 , -1.5250947 ],
       [-1.3787905 ,  1.3881468 ],
       [-0.45730427,  0.5417524 ],
       [-0.5810372 ,  0.7226909 ],
       [-0.9454785 ,  1.0208609 ],
       [ 1.3025333 , -1.4313099 ],
       [-0.52752346,  0.68885744],
       [ 1.3655891 , -1.4621383 ],
       [-1.1327679 ,  1.16

In [48]:
predictions.predictions

array([[-0.78361255,  0.7879547 ],
       [ 1.3924159 , -1.4180576 ],
       [ 1.1251961 , -1.2252665 ],
       [-0.12519355,  0.17314416],
       [-1.2161793 ,  1.3727537 ],
       [-1.3480221 ,  1.3811079 ],
       [ 1.3010815 , -1.3765541 ],
       [-1.0024374 ,  1.1217505 ],
       [ 1.3942534 , -1.5273594 ],
       [ 0.99308556, -0.96962404],
       [-1.2071494 ,  1.3180724 ],
       [-0.0117905 , -0.05663886],
       [ 0.39861912, -0.5109151 ],
       [-0.23184982,  0.266297  ],
       [-1.0627222 ,  1.201213  ],
       [-0.6160534 ,  0.6682716 ],
       [ 1.0276047 , -1.0495312 ],
       [-0.9789389 ,  1.1112759 ],
       [-0.21778204,  0.30491042],
       [ 1.3858902 , -1.5250947 ],
       [-1.3787905 ,  1.3881468 ],
       [-0.45730427,  0.5417524 ],
       [-0.5810372 ,  0.7226909 ],
       [-0.9454785 ,  1.0208609 ],
       [ 1.3025333 , -1.4313099 ],
       [-0.52752346,  0.68885744],
       [ 1.3655891 , -1.4621383 ],
       [-1.1327679 ,  1.1655061 ],
       [ 1.2053504 ,

Get predicted classes.

In [49]:
# https://numpy.org/doc/2.2/reference/generated/numpy.argmax.html
# Returns the index of the maximum value along an axis.
# Axis=1 to find the index of the maximum value in each row of the 2D array.
# In this case, that's the class with the highest logit, i.e., the predicted class.
predicted_classes = np.argmax(predictions.predictions, axis=1)
predicted_classes

array([1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0])

Add predictions to the original `new` dataframe.

In [50]:
new['predicted_class'] = predicted_classes

In [51]:
new.columns

Index(['idx', 'label', 'label_text', 'text_original', 'label_domain_text',
       'label_subcat_text', 'text_preceding', 'text_following', 'manifesto_id',
       'doc_id', 'country_name', 'date', 'party', 'cmp_code_hb4', 'cmp_code',
       'label_subcat_text_simple', 'text', 'predicted_class'],
      dtype='object')

In [52]:
new['predicted_class'].value_counts()

,count
predicted_class,
0,48
1,38


Since our "`new`" data actually had labels, we can compare the predictions with the labels, which wouldn't be the case in an actual research project.

In [53]:
pd.crosstab(new['label'], new['predicted_class'], margins=True)

predicted_class,0,1,All
label,,,
0,35,2,37
1,13,36,49
All,48,38,86


## Answer research question

In [54]:
pd.crosstab(new['country_name'], new['predicted_class'], normalize='index')

predicted_class,0,1
country_name,,
Australia,0.000000,1.000000
Ireland,0.666667,0.333333
New Zealand,0.625000,0.375000
United Kingdom,0.375000,0.625000
United States,0.722222,0.277778


In [55]:
pd.crosstab(new['country_name'], new['predicted_class'], margins=True)

predicted_class,0,1,All
country_name,,,
Australia,0,6,6
Ireland,4,2,6
New Zealand,25,15,40
United Kingdom,6,10,16
United States,13,5,18
All,48,38,86


## Continue learning

This notebook includes a lot of URLs that you can go to to learn more about the code. You can also consult these websites:
- https://huggingface.co/docs/transformers/en/training
- https://medium.com/@hassaanidrees7/fine-tuning-transformers-techniques-for-improving-model-performance-4b4353e8ba93
- https://huggingface.co/learn/llm-course/chapter3/3

This notebook uses code produced by ChatGPT. That's okay to do as long as you consider the privacy, security, and intellectual property implications, as well as understand the code. We do have a [workshop on writing effective prompts for coding with LLMs](https://github.com/nuitrcs/promptEngineering).